<a href="https://colab.research.google.com/github/irajamuller/introducao-tecnologias-quanticas-2025/blob/main/Arxiv_An%C3%A1lise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd
import requests

url = 'https://raw.githubusercontent.com/irajamuller/introducao-tecnologias-quanticas-2025/refs/heads/main/artigos_arxiv_quantum_computing.json'

response = requests.get(url)
response.raise_for_status()

data = response.json()
# Dataset artigo -> linha
df = pd.json_normalize(data)
df = df[~((df['ano'] == 2021) & (df['ano'] != 'N/A'))]

In [ ]:
# Total de artigos
len(df)

9704

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9958 entries, 0 to 9957
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titulo           9958 non-null   object
 1   autores          9958 non-null   object
 2   abstract         9958 non-null   object
 3   keywords         9958 non-null   object
 4   categorias       9958 non-null   object
 5   ano              9958 non-null   object
 6   journal_revista  9958 non-null   object
dtypes: object(7)
memory usage: 544.7+ KB


In [ ]:
# Número de artigos por ano
df_ano = df[df['ano'] != 'N/A']
df_ano = df_ano['ano'].value_counts()
df_ano

,count
ano,
2025,3168
2024,2807
2023,2054
2022,1674


In [ ]:
# Por categorias
df_exploded = (
    df
    .explode('categorias')
    .reset_index(drop=True)
)
# Exclui-se a primeira categoria
df_exploded['categorias'].value_counts().sort_values(ascending=False).head(11)

,count
categorias,
quant-ph,9052
cs.LG,697
cs.ET,544
cond-mat.mes-hall,434
cs.CR,405
cond-mat.str-el,375
cs.AI,347
physics.chem-ph,259
physics.optics,255


In [ ]:
# Geração das palavras-chave para nuvem de palavra# Por categorias
df_exploded = (
    df
    .explode('keywords')
    .reset_index(drop=True)
)
# Exclui-se a primeira categoria
df_exploded = df_exploded[(df_exploded['keywords'] != 'quantum computing') & (df['keywords'] != 'N/A')]
df_exploded['keywords'] = df_exploded['keywords'].str.lower()

#df_exploded['keywords'].to_csv('df_exploded.csv', index=False, header=False)
df_filtered = df_exploded['keywords'].value_counts().sort_values(ascending=False)

df_filtered.to_csv('df_exploded.csv', header=False)

7102


In [ ]:
# Revistas
df_filtered = df[df['journal_revista'] != 'N/A'].copy()
df_filtered['journal_revista'] = df_filtered['journal_revista'].str.lower().replace(r'\(\d{4}\)', '', regex=True).str.strip()
df_filtered['journal_revista'] = df_filtered['journal_revista'].str.replace(r'\d{4}', '', regex=True).str.strip().copy()
df_filtered['journal_revista'] = df_filtered['journal_revista'].str.replace(r'[\d(),;]', '', regex=True).str.strip().copy()
#df_filtered['journal_revista'].head(20)
print(len(df_filtered['journal_revista']), len(df_filtered['journal_revista']) / len(df) * 100)
df_filtered['journal_revista'].value_counts().sort_values(ascending=False).head(11
                                                                                )

2878 29.657873042044518


,count
journal_revista,
quantum,304
phys. rev. a,243
phys. rev. research,141
phys. rev. lett.,128
prx quantum,123
phys. rev. applied,88
quantum sci. technol.,58
phys. rev. b,54
physical review a,36


In [ ]:
# Por categorias
df_exploded = (
    df
    .explode('categorias')
    .reset_index(drop=True)
)

df_exploded['categorias'].value_counts().sort_values(ascending=False).head(6)




,count
categorias,
quant-ph,9052
cs.LG,697
cs.ET,544
cond-mat.mes-hall,434
cs.CR,405
cond-mat.str-el,375


In [ ]:
# cs.LG, cs.ET, cond-mat.mes-hall, cs.CR, cond-mat.str-el

df_exploded = df_exploded[df_exploded['categorias'].isin(['cs.LG', 'cs.ET', 'cond-mat.mes-hall', 'cs.CR', 'cond-mat.str-el'])]

# Exclui-se a primeira categoria
df_grouped = df_exploded.groupby(['categorias', 'ano']).size().reset_index(name='contagem')

# Ordenar pelo ano e pela contagem (opcional)
df_grouped = df_grouped.sort_values(by=['ano', 'contagem'], ascending=[True, False])

df_grouped.to_csv('df_category.csv', header=False)